<a href="https://colab.research.google.com/github/Vanpersact/hello-world/blob/main/BarChartRace_Manifestacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Bar Chart Race — Manifestações por período (diário/mensal)

Gera um **ranking animado** a partir de uma base em formato longo com:

- `data` (YYYY-MM-DD ou YYYY-MM)
- `categoria` (ex.: Linha, Tipo)
- `valor` (quantidade no período)

Ajuste as variáveis na seção **Configuração** e rode as células.


In [53]:

# Bibliotecas
import os, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
from matplotlib.cm import get_cmap

pd.set_option("display.max_rows", 10)



## 1) Configuração


In [54]:

# >>>>>>> edite aqui <<<<<<<
ARQUIVO_DADOS = "Manifestacoes.csv"   # ex.: "manifestacoes.csv"  (deixe None para usar um exemplo sintético)
FREQ = "M"            # "D" = diário | "M" = mensal
TOP_N = 15            # Top N do ranking
STEPS_PER_PERIOD = 10 # suavização (maior = mais liso e mais pesado)
FPS = 20              # frames por segundo
SAIDA = "bar_chart_race_manifestacoes.gif"  # .gif (Pillow) ou .mp4 (se FFMPEG disponível)

# Se suas colunas tiverem outros nomes, ajuste aqui:
COL_DATA = "data"
COL_CATEGORIA = "categoria"
COL_VALOR = "valor"



## 2) Carregamento da base


In [69]:

def carregar_base(arquivo):
    #import pandas as pd
    if arquivo.lower().endswith(".csv"):
        df = pd.read_csv(arquivo, sep=None, engine="python", encoding="utf-8-sig")
    else:
        df = pd.read_excel(arquivo)
    df.rename(columns={c: c.strip().lower() for c in df.columns}, inplace=True)

    # mapear seus nomes (DIA/MOTIVO/QTDE) -> (data/categoria/valor)
    mapa = {
        "data":      ["data","dia","mes_ano","mes","date"],
        "categoria": ["categoria","motivo","linha","tipo","classe"],
        "valor":     ["valor","qtde","qtd","quantidade","volume"]
    }
    for alvo, cands in mapa.items():
        if alvo not in df.columns:
            for c in cands:
                if c in df.columns:
                    df.rename(columns={c: alvo}, inplace=True)
                    break

    faltando = [c for c in ["data","categoria","valor"] if c not in df.columns]
    if faltando:
        raise ValueError(f"Colunas ausentes: {faltando}. Achadas: {list(df.columns)}")

    df["data"] = pd.to_datetime(df["data"], errors="coerce", dayfirst=True)
    df = df.dropna(subset=["data","categoria","valor"]).copy()
    df["categoria"] = df["categoria"].astype(str).str.strip()
    df["valor"] = pd.to_numeric(df["valor"], errors="coerce").fillna(0).astype(int)
    return df

# >>> carregue de fato a base <<<
df = carregar_base(ARQUIVO_DADOS)
print(df.head())
print(df.dtypes)

        data        categoria  valor
0 2020-01-01     FISCALIZAÇÃO      1
1 2020-01-01    PROCEDIMENTOS      1
2 2020-01-01  TEMPO DE VIAGEM      1
3 2020-01-01             TREM      2
4 2020-02-01   ACESSIBILIDADE      3
data         datetime64[ns]
categoria            object
valor                 int64
dtype: object



## 3) Preparação do painel


In [70]:

def preparar_painel(df, freq="M"):
    df = df.copy()
    if freq not in ("D","M"):
        raise ValueError("freq deve ser 'D' (diário) ou 'M' (mensal).")
    if freq == "M":
        df["periodo"] = df[COL_DATA].dt.to_period("M").dt.to_timestamp()
    else:
        df["periodo"] = df[COL_DATA].dt.floor("D")
    agg = (df.groupby(["periodo", COL_CATEGORIA], as_index=False)[COL_VALOR].sum()
             .sort_values(["periodo", COL_VALOR], ascending=[True, False]))
    painel = agg.pivot_table(index="periodo", columns=COL_CATEGORIA, values=COL_VALOR, fill_value=0)
    painel = painel.sort_index()
    return painel, agg

painel, agg = preparar_painel(df, FREQ)
painel.head()


categoria,ACESSIBILIDADE,CANAIS DE INTERAÇÃO,CUSTO,EMPREGADOS,ESTAÇÃO,EXPECTATIVA DO SERVIÇO,FAIXA DE DOMÍNIO,FISCALIZAÇÃO,FLUXO DE PASSAGEIROS,INFORMAÇÃO,INTEGRAÇÕES,LIGAÇÃO,OUVIDORIA,PROCEDIMENTOS,SISTEMA DE INFORMAÇÃO,TEMPO DE VIAGEM,TREM
periodo,,,,,,,,,,,,,,,,,
2020-01-01,11.0,15.0,1.0,45.0,38.0,15.0,24.0,39.0,48.0,29.0,5.0,0.0,0.0,51.0,47.0,189.0,31.0
2020-02-01,18.0,16.0,5.0,83.0,48.0,32.0,17.0,48.0,74.0,41.0,3.0,0.0,0.0,64.0,57.0,293.0,56.0
2020-03-01,16.0,14.0,7.0,71.0,57.0,29.0,29.0,34.0,80.0,23.0,6.0,0.0,0.0,60.0,43.0,249.0,37.0
2020-04-01,10.0,20.0,5.0,70.0,62.0,35.0,24.0,62.0,85.0,30.0,6.0,0.0,0.0,68.0,65.0,310.0,25.0
2020-05-01,14.0,11.0,3.0,54.0,57.0,32.0,19.0,58.0,80.0,21.0,2.0,0.0,0.0,71.0,50.0,275.0,39.0



## 4) Função do bar chart race


In [79]:
def bar_chart_race(
    painel,
    top_n=10,
    steps_per_period=30,        # mais passos = transição mais suave
    period_ms=3000,             # 2s por período (mês)
    saida="bar_chart_race.mp4",
    titulo=None,
    period_fmt="%Y-%m",
    fixed_max=1000,            # True = usa máximo global; número = usa esse valor; False = dinâmico
    fixed_max_pad=0.10,         # folga quando fixed_max=True
    figsize=(11, 6),
    dpi=110,
    left_margin=0.35,
    truncate_labels_at=None
):
    # ---------- saneamento numérico ----------
    painel = painel.copy()
    for c in painel.columns:
        painel[c] = pd.to_numeric(painel[c], errors="coerce")
    painel = painel.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    periods = painel.index.to_list()
    cats = list(painel.columns)

    # ---------- paleta de cores estável ----------
    cmap = plt.get_cmap("tab20")
    cores = {c: cmap(i % cmap.N) for i, c in enumerate(sorted(cats))}

    # encurta rótulos opcionalmente
    def _short(s):
        s = str(s)
        if truncate_labels_at and len(s) > truncate_labels_at:
            return s[:truncate_labels_at - 1] + "…"
        return s
    label_map = {c: _short(c) for c in cats}

    # ---------- easing para suavizar (smoothstep) ----------
    ###def ease_smoothstep(t):
    ###    # t em [0,1] -> suaviza in/out
    ###    return t * t * (3 - 2 * t)
    def ease_smootherstep(t):
        # 6t^5 - 15t^4 + 10t^3
        return t**3 * (t * (6*t - 15) + 10)
    # ---------- frames com interpolação ----------
    frames = []
    for i in range(len(periods) - 1):
        a, b = periods[i], periods[i + 1]
        va = painel.loc[a]
        vb = painel.loc[b]
        uni = sorted(set(va.index) | set(vb.index))
        va = va.reindex(uni).fillna(0.0)
        vb = vb.reindex(uni).fillna(0.0)

        for s in range(steps_per_period):
            ###t0 = s / float(steps_per_period)
            ###t  = ease_smoothstep(t0)
            ###v = va * (1 - t) + vb * t
            t0 = s / float(steps_per_period)
            t  = ease_smootherstep(t0)
            v = va * (1 - t) + vb * t
            frames.append((a, v))

    # último quadro no valor final
    frames.append((periods[-1], painel.iloc[-1].reindex(cats).fillna(0.0)))

    # ---------- figura ----------
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    fig.subplots_adjust(left=left_margin, right=0.96, top=0.90, bottom=0.12)
    ax.set_facecolor("#f8f8f8"); fig.patch.set_facecolor("#f8f8f8")

    title_obj = fig.text(0.5, 0.96, titulo or "Bar Chart Race",
                         ha="center", va="bottom", fontsize=16, weight="bold")
    date_obj  = fig.text(0.50, 0.92, "", ha="center", va="top",
                         fontsize=22, weight="bold")

    # ---------- máximo global robusto ----------
    def safe_max_df(df_like):
        arr = np.asarray(df_like.replace([np.inf, -np.inf], np.nan), dtype=float)
        if arr.size == 0:
            return 0.0
        m = np.nanmax(arr)
        return 0.0 if np.isnan(m) else float(m)

    if isinstance(fixed_max, (int, float)):
        x_max_global = float(fixed_max)
    elif fixed_max is True:
        x_max_global = safe_max_df(painel) * (1.0 + fixed_max_pad)
    else:
        x_max_global = None  # dinâmico

    def fmt_val(x):  # 1.234 -> "1.234"
        return f"{int(round(float(x))):,}".replace(",", ".")

    def draw_frame(frame_data):
        periodo, valores = frame_data

        ax.clear()
        ax.set_facecolor("#f8f8f8")
        ax.grid(axis="x", zorder=0, linestyle="--", alpha=0.3)

        valores = pd.to_numeric(valores, errors="coerce").fillna(0.0)

        ###top = valores.sort_values(ascending=False).head(top_n)[::-1]
        top = valores.sort_values(ascending=False, kind="mergesort").head(top_n)[::-1]
        ax.barh([label_map[c] for c in top.index], top.values,
                color=[cores[c] for c in top.index], zorder=3)

        for cat, val in zip([label_map[c] for c in top.index], top.values):
            ax.text(val, cat, f" {fmt_val(val)}", va="center", ha="left", fontsize=11)

        # eixo X fixo ou dinâmico
        if x_max_global is not None:
            ax.set_xlim(0, max(1.0, x_max_global))
        else:
            mloc = safe_max_df(valores)
            ax.set_xlim(0, max(1.0, mloc) * 1.15)

        ax.set_xlabel("Quantidade")
        ax.set_ylabel("Categoria")
        title_obj.set_text(titulo or "Bar Chart Race")

        ###try:
        ###    date_obj.set_text(pd.to_datetime(periodo).strftime(period_fmt))
        ###except Exception:
        ###    date_obj.set_text(str(periodo))

        try:
            dt = pd.to_datetime(periodo)
            meses_pt = ["janeiro","fevereiro","março","abril","maio","junho",
                        "julho","agosto","setembro","outubro","novembro","dezembro"]
            date_obj.set_text(f"{dt.year}-{meses_pt[dt.month-1].capitalize()}")
        except Exception:
            date_obj.set_text(str(periodo))

    # ---------- temporização ----------
    # cada período = period_ms; com 'steps_per_period' subquadros
    interval_ms = float(period_ms) / max(1, steps_per_period)
    fps_final = 2000.0 / interval_ms  # frames/segundo do arquivo gerado

    ani = animation.FuncAnimation(
        fig, draw_frame, frames=frames,
        interval=interval_ms, blit=False, repeat=False
    )

    if saida.lower().endswith(".gif"):
        ani.save(saida, writer=PillowWriter(fps=fps_final))
    else:
        try:
            ani.save(saida, fps=fps_final)
        except Exception as e:
            print("FFMPEG não disponível. Salve como .gif ou instale FFMPEG. Erro:", e)

    plt.close(fig)
    return saida

## 5) Geração do arquivo para download

In [80]:

titulo = f"Reclamações — Ranking por {('dia' if FREQ=='D' else 'mês')} (Top {TOP_N})"
arquivo_saida = bar_chart_race(
    painel,                 # seu pivot período x categoria com contagens reais
    top_n=10,
    steps_per_period=30,    # 10 passos (suave)
    period_ms=3000,         # 2 segundos por mês
    saida="ranking_mensal.mp4",
    titulo="Reclamações — Ranking por mês (Top 10)",
    period_fmt="%Y-%m",     # exibe "2021-03"
    fixed_max=1000,         # usa máximo global do painel
    fixed_max_pad=0.10      # 10% de folga
)
arquivo_saida


'ranking_mensal.mp4'

In [22]:
# VERSÃO ANTERIOR
from matplotlib.cm import get_cmap

def bar_chart_race(painel, top_n=15, steps_per_period=10, fps=20, saida="bar_chart_race.gif", titulo=None):
    painel = painel.copy().astype(float).fillna(0.0)
    periods = painel.index.to_list()
    cats = list(painel.columns)
    cmap = get_cmap("tab20")
    cores = {c: cmap(i % 20) for i, c in enumerate(sorted(cats))}
    frames = []
    for i in range(len(periods)-1):
        a, b = periods[i], periods[i+1]
        va = painel.loc[a]
        vb = painel.loc[b]
        union_cats = sorted(set(va.index) | set(vb.index))
        va = va.reindex(union_cats).fillna(0)
        vb = vb.reindex(union_cats).fillna(0)
        for s in range(steps_per_period):
            t = s / float(steps_per_period)
            v = va*(1-t) + vb*t
            frames.append((a, v))
    frames.append((periods[-1], painel.iloc[-1].reindex(cats).fillna(0)))
    fig, ax = plt.subplots(figsize=(11, 6))
    fig.subplots_adjust(left=0.35, right=0.96, top=0.90, bottom=0.12)  # mais espaço à esquerda
    ax.set_facecolor("#f8f8f8")
    fig.patch.set_facecolor("#f8f8f8")
    ax.grid(axis="x", zorder=0, linestyle="--", alpha=0.3)
    title_text = ax.text(0.5, 1.05, titulo or "Bar Chart Race", transform=ax.transAxes,
                         ha="center", va="bottom", fontsize=16, weight="bold")
    date_text = ax.text(0.98, 0.95, "", transform=ax.transAxes, ha="right", va="top",
                        fontsize=12, family="monospace")
    def draw_frame(frame_data):
        periodo, valores = frame_data
        ax.clear()
        ax.set_facecolor("#f8f8f8")
        ax.grid(axis="x", zorder=0, linestyle="--", alpha=0.3)
        top = valores.sort_values(ascending=False).head(top_n)[::-1]
        ax.barh(top.index, top.values, color=[cores[c] for c in top.index], zorder=3)
        for cat, val in zip(top.index, top.values):
            ax.text(val, cat, f" {int(val):,}".replace(",", "."), va="center", ha="left", fontsize=11)
        ax.set_xlim(0, max(1, valores.max())*1.15)
        ax.set_xlabel("Quantidade")
        ax.set_ylabel("Categoria")
        title_text.set_text(titulo or "Bar Chart Race")
        date_text.set_text(pd.to_datetime(periodo).strftime("%Y-%m-%d"))
    ani = animation.FuncAnimation(fig, draw_frame, frames=frames, interval=1000/fps, blit=False, repeat=False)
    if saida.lower().endswith(".gif"):
        ani.save(saida, writer=PillowWriter(fps=fps))
    else:
        try:
            ani.save(saida, fps=fps)
        except Exception as e:
            print("FFMPEG não disponível. Salve como .gif ou instale FFMPEG. Erro:", e)
    plt.close(fig)
    return saida



## 5) Gerar o GIF/MP4


/tmp/ipython-input-3662319787.py:30: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = get_cmap("tab20")


'bar_chart_race_manifestacoes.gif'